In [12]:
import sys
print(sys.executable)
# Verify kernal path
import numpy as np
import pandas as pd
import os
import requests
from dotenv import load_dotenv
load_dotenv()


prices = pd.read_parquet("../data/transformed/validated_prices_2022-08-02-2024-08-02.parquet") # reads in parquet-file in specified path and returns its data in a dataframe
prices


/Users/pravachanpatra/Documents/PYTHON/AI_ML_DL/Stock_Price_Predictor/venv/bin/python


,volume,vw_avr_price,open_price,close_price,high_price,low_price,num_transactions,datetime,price_diff,volume_sma_5,volume_change,SMA_5,SMA_20,EMA_5,EMA_20,vol_5,daily_return
0,5.669698e+07,164.743200,163.210000,165.35,165.850000,163.00,491310.000000,2022-08-02 00:00:00,0.000000,5.694134e+07,0.000000,165.318000,169.103000,165.350000,165.350000,0.071554,0.000000
1,5.669698e+07,164.743200,163.210000,165.35,165.850000,163.00,491310.000000,2022-08-03 00:00:00,0.000000,5.694134e+07,0.000000,165.318000,169.103000,165.350000,165.350000,0.071554,0.000000
2,5.669698e+07,164.743200,163.210000,165.35,165.850000,163.00,491310.000000,2022-08-04 00:00:00,0.000000,5.694134e+07,0.000000,165.318000,169.103000,165.350000,165.350000,0.071554,0.000000
3,5.669698e+07,164.743200,163.210000,165.35,165.850000,163.00,491310.000000,2022-08-05 04:00:00,0.000000,5.694134e+07,0.000000,165.318000,169.103000,165.350000,165.350000,0.071554,0.000000
4,5.791877e+07,165.126767,164.263333,165.19,166.503333,163.40,507545.666667,2022-08-06 00:00:00,-0.160000,5.694134e+07,0.021549,165.318000,169.103000,165.296667,165.334762,0.071554,-0.000968
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
727,3.515373e+07,218.131900,216.960000,218.24,219.300000,215.75,604680.000000,2024-07-29 04:00:00,0.093333,4.008290e+07,-0.042438,217.978000,225.414500,218.726415,221.994869,0.292077,0.000428
728,4.068162e+07,218.405900,219.190000,218.80,220.325000,216.12,584305.000000,2024-07-30 04:00:00,0.560000,3.812887e+07,0.157249,218.240000,224.705500,218.750943,221.690596,0.329983,0.002566
729,4.842297e+07,222.344100,221.440000,222.08,223.820000,220.63,668833.000000,2024-07-31 04:00:00,3.280000,3.984794e+07,0.190291,219.064000,224.431000,219.860629,221.727682,1.710839,0.014991
730,6.112524e+07,219.477300,224.370000,218.36,224.480000,217.02,876046.000000,2024-08-01 04:00:00,-3.720000,4.441905e+07,0.262319,219.125333,223.822000,219.360419,221.406950,1.670571,-0.016751


In [13]:
def get_cutoff_indicies(data, n_previous_days, step_size):
    stop_position = len(data)-1

    first_indx = 0
    last_indx = n_previous_days+1  # target index
    indicies = []

    while last_indx <= stop_position:
        indicies.append((first_indx, last_indx))

        first_indx += step_size
        last_indx += step_size
    
    return indicies

n_previous_days = 12
step_size = 1
indicies = get_cutoff_indicies(prices, n_previous_days, step_size)
indicies[0:10]

[(0, 13),
 (1, 14),
 (2, 15),
 (3, 16),
 (4, 17),
 (5, 18),
 (6, 19),
 (7, 20),
 (8, 21),
 (9, 22)]

In [14]:
def transform_ts_data_into_features_target(prices, n_previous_days, step_size):
    # Get indices for slicing
    indicies = get_cutoff_indicies(prices, n_previous_days, step_size)

    # each element is an example, each example contains the previous-n-days, each day contains its features values
    features = []
    targets = []

    # iterate every indicies pair, number of indicies is number of examples
    for start_idx, end_idx in indicies:
        # extract feature values for previous days (d1, d2, ..., dn)
        # prices extract days-rows from start-indx of example to end-indx not inclusive because its the target, this is an example
        feature_values = prices.iloc[start_idx:end_idx]
        
        # convert feature values to numerical only
        feature_values = feature_values.apply(pd.to_numeric, errors='coerce')
        
        # add the days-rows of cur indx-pair to features which is an example
        features.append(feature_values.values)
        
        # extract target value for the next day which is close-price, the end-index of window
        target_value = prices.iloc[end_idx]['close_price']
        targets.append(target_value)  # add target value of example to targets

    # convert to numpy arrays
    features = np.array(features, dtype=np.float32)
    targets = np.array(targets, dtype=np.float32)

    # flatten into 2D since 3-dimensional cannot we fed into model 
    # features = [d1f1, d1f2, d1f3, d2f1, d2f2, d2f3]
    n_previous_days = features.shape[1]
    n_features = features.shape[2]
    features = features.reshape(features.shape[0], n_previous_days * n_features)

    return features, targets


features, targets = transform_ts_data_into_features_target(prices, n_previous_days, step_size)
print(f"Num-features-in-each-day: {len(prices.columns)-1}")
print(f"N-previous-days: {n_previous_days}")
print(f"Step-size: {step_size}")
print(f'Features shape: {features.shape}')  # Should be (number_of_examples, n_previous_days, n_features)
print(f'Targets shape: {targets.shape}')    # Should be (number_of_examples,)

Num-features-in-each-day: 16
N-previous-days: 12
Step-size: 1
Features shape: (719, 221)
Targets shape: (719,)


In [15]:
tabular_data = pd.DataFrame(features)
tabular_data["target_close_price_next_day"] = pd.DataFrame(targets)
tabular_data
# each column is a dayx-featurey

,0,1,2,3,4,5,6,7,8,9,...,212,213,214,215,216,217,218,219,220,target_close_price_next_day
0,56696984.0,164.743195,163.210007,165.350006,165.850006,163.000000,491310.00000,1.659398e+18,0.000000,56941340.0,...,0.363333,63496620.0,-0.073343,171.024002,169.102997,170.947968,167.599106,2.005173,0.002107,173.190002
1,56696984.0,164.743195,163.210007,165.350006,165.850006,163.000000,491310.00000,1.659485e+18,0.000000,56941340.0,...,0.363333,60280852.0,-0.079148,171.813995,169.102997,171.695312,168.131577,1.902056,0.002102,173.029999
2,56696984.0,164.743195,163.210007,165.350006,165.850006,163.000000,491310.00000,1.659571e+18,0.000000,56941340.0,...,-0.160000,60127840.0,0.042250,172.722000,169.102997,172.140213,168.598099,0.441202,-0.000924,174.550003
3,56696984.0,164.743195,163.210007,165.350006,165.850006,163.000000,491310.00000,1.659672e+18,0.000000,56941340.0,...,1.520000,62428372.0,0.410894,173.212006,169.102997,172.943466,169.164948,0.795744,0.008785,174.149994
4,57918768.0,165.126770,164.263336,165.190002,166.503326,163.399994,507545.65625,1.659744e+18,-0.160000,56941340.0,...,-0.400000,62208356.0,-0.216891,173.549332,169.102997,173.345642,169.639709,0.755515,-0.002292,171.520004
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
714,38468544.0,234.299194,235.000000,234.820007,236.270004,232.330002,623906.00000,1.721102e+18,0.420000,52547780.0,...,0.093333,44989644.0,-0.040710,218.037994,225.936493,218.969620,222.390121,0.377636,0.000428,218.240005
715,55878904.0,228.839600,229.449997,228.880005,231.459900,226.639999,811024.00000,1.721189e+18,-5.940000,53399272.0,...,0.093333,40082904.0,-0.042438,217.977997,225.414505,218.726410,221.994873,0.292077,0.000428,218.800003
716,64346032.0,225.181000,230.279999,224.179993,230.440002,222.270004,844538.00000,1.721275e+18,-4.700000,55351372.0,...,0.560000,38128876.0,0.157249,218.240005,224.705505,218.750946,221.690598,0.329983,0.002566,222.080002
717,48020040.0,224.735596,224.820007,224.309998,226.800003,223.274994,612739.00000,1.721362e+18,0.130000,53445452.0,...,3.280000,39847940.0,0.190291,219.063995,224.431000,219.860626,221.727676,1.710839,0.014991,218.360001


In [16]:
tabular_data.to_parquet("../data/transformed/tabular_prices_2022-08-02-2024-08-02.parquet")

/Users/pravachanpatra/Documents/PYTHON/AI_ML_DL/Stock_Price_Predictor/venv/lib/python3.12/site-packages/pandas/io/parquet.py:190: UserWarning: The DataFrame has column names of mixed type. They will be converted to strings and not roundtrip correctly.
  table = self.api.Table.from_pandas(df, **from_pandas_kwargs)


In [80]:
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error



model = GradientBoostingRegressor(n_estimators=100, random_state=42)
model.fit(features, targets)

predictions = model.predict(features)
mse = mean_squared_error(targets, predictions)
print(f'Mean Squared Error: {mse}')
print(len(features[0]))

Mean Squared Error: 0.9815958805094114
221


In [65]:
from sklearn.ensemble import RandomForestRegressor

rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(features, targets)

# Make predictions and calculate MSE
y_pred = rf_model.predict(features)
mse = mean_squared_error(targets, y_pred)
print(f'Random Forest MSE: {mse}')

Random Forest MSE: 0.7365136091476493
